<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [2]:
!pip3 install beautifulsoup4
!pip3 install requests

In [3]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [4]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [6]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [7]:

response = requests.get(static_url)


if response.status_code == 200:
   
    soup = BeautifulSoup(response.content, 'html.parser')
    print("Successfully fetched and parsed the page!")
    
    
    tables = soup.find_all('table', {'class': 'wikitable'})

  
    for table in tables:
        headers = [header.text.strip() for header in table.find_all('th')]
        rows = table.find_all('tr')[1:]  # Skip the header row

     
        print("Headers:", headers)

      
        for row in rows:
            cells = row.find_all('td')
            cell_data = [cell.text.strip() for cell in cells]
            print(cell_data)



Successfully fetched and parsed the page!
Headers: ['Flight No.', 'Date andtime (UTC)', 'Version,Booster [b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '1', '2', '3', '4', '5', '6', '7']
['4 June 2010,18:45', 'F9 v1.0[7]B0003.1[8]', 'CCAFS,SLC-40', 'Dragon Spacecraft Qualification Unit', '', 'LEO', 'SpaceX', 'Success', 'Failure[9][10](parachute)']
['First flight of Falcon 9 v1.0.[11] Used a boilerplate version of Dragon capsule which was not designed to separate from the second stage.(more details below) Attempted to recover the first stage by parachuting it into the ocean, but it burned up on reentry, before the parachutes even deployed.[12]']
['8 December 2010,15:43[13]', 'F9 v1.0[7]B0004.1[8]', 'CCAFS,SLC-40', 'Dragon demo flight C1(Dragon C101)', '', 'LEO (ISS)', 'NASA (COTS)\nNRO', 'Success[9]', 'Failure[9][14](parachute)']
['Maiden flight of Dragon capsule, consisting of over 3 hours of testing thruster maneuvering and r

Create a `BeautifulSoup` object from the HTML `response`


In [6]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
if response.status_code == 200:
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    print("Successfully fetched and parsed the page!")

Successfully fetched and parsed the page!


Print the page title to verify if the `BeautifulSoup` object was created properly 


In [8]:
# Use soup.title attribute
print("Page title:", soup.title.string)

Page title: List of Falcon 9 and Falcon Heavy launches - Wikipedia


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [1]:

html_tables = soup.find_all('table', {'class': 'wikitable'})
    

print(f"Number of tables found: {len(html_tables)}")
    
   
if len(html_tables) > 0:
        first_table = html_tables[0]
        headers = [header.text.strip() for header in first_table.find_all('th')]
        print("Column names:", headers)



NameError: name 'soup' is not defined

Starting from the third table is our target table contains the actual launch records.


In [12]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a><sup class="reference" id="cite_ref-booster_11-2"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-2"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
<tr>
<th rowspan="2" scope="row" style="text-align:center;">14
</th>
<td>

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [14]:
column_names = []

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names
if len(html_tables) >= 3:
        first_launch_table = html_tables[2]
        
        # Define a function to extract column names from header
def extract_column_from_header(header):
    return header.text.strip()
        
        # Extract column names from the third table
column_names = []
for header in first_launch_table.find_all('th'):
    name = extract_column_from_header(header)
    if name is not None and len(name) > 0:
        column_names.append(name)
        
print("Column names:", column_names)
 

Column names: ['Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '14', '15', '16', '17', '18', '19', '20']


Check the extracted column names


In [15]:
print(column_names)

['Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '14', '15', '16', '17', '18', '19', '20']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [20]:
launch_dict= dict.fromkeys(column_names)
if 'Date and time ( )' in launch_dict:
        del launch_dict['Date and time ( )']
#
        
# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]
def date_time(cell):
            return cell.text.strip().split()

def booster_version(cell):
            return cell.text.strip()

def get_mass(cell):
            return cell.text.strip()

def landing_status(cell):
            return cell.text.strip()
        
        

In [93]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define the URL
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Fetch the HTML content using requests.get()
response = requests.get(static_url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    print("Successfully fetched and parsed the page!")
    
    # Print the page title to verify the BeautifulSoup object
    print("Page title:", soup.title.string)
    
    # Find all tables on the wiki page
    html_tables = soup.find_all('table', {'class': 'wikitable'})
    
    # Extract and print the content of the third table
    if len(html_tables) >= 3:
        first_launch_table = html_tables[2]
        
        # Define a function to extract column names from header
        def extract_column_from_header(header):
            return header.text.strip()
        
        # Extract column names from the third table
        column_names = []
        for header in first_launch_table.find_all('th'):
            name = extract_column_from_header(header)
            if name is not None and len(name) > 0:
                column_names.append(name)
        
        print("Column names:", column_names)
        
        # Create an empty dictionary with keys from the extracted column names
        launch_dict = dict.fromkeys(column_names)
        
        # Remove an irrelevant column
        if 'Date and time ( )' in launch_dict:
            del launch_dict['Date and time ( )']
        
        # Initialize the launch_dict with each value to be an empty list
        launch_dict['Flight No.'] = []
        launch_dict['Launch site'] = []
        launch_dict['Payload'] = []
        launch_dict['Payload mass'] = []
        launch_dict['Orbit'] = []
        launch_dict['Customer'] = []
        launch_dict['Launch outcome'] = []
        launch_dict['Version Booster'] = []
        launch_dict['Booster landing'] = []
        launch_dict['Date'] = []
        launch_dict['Time'] = []

        def date_time(cell):
            return cell.text.strip().split()

        def booster_version(cell):
            return cell.text.strip()

        def get_mass(cell):
            return cell.text.strip()

        def landing_status(cell):
            return cell.text.strip()
        
        extracted_row = 0
        # Extract each table
        for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
            # Get table row
            for rows in table.find_all("tr"):
                # Check to see if first table heading is a number corresponding to launch a number 
                if rows.th:
                    if rows.th.string:
                        flight_number = rows.th.string.strip()
                        flag = flight_number.isdigit()
                else:
                    flag = False
                # Get table element 
                row = rows.find_all('td')
                # If it is a number save cells in a dictionary 
                if flag:
                    extracted_row += 1
                    # Flight Number value
                    launch_dict['Flight No.'].append(flight_number)
                    
                    datatimelist = date_time(row[0])
                    
                    # Date value
                    date = datatimelist[0].strip(',')
                    launch_dict['Date'].append(date)
                    
                    # Time value
                    time = datatimelist[1]
                    launch_dict['Time'].append(time)
                    
                    # Booster version
                    bv = booster_version(row[1])
                    if not bv:
                        bv = row[1].a.string if row[1].a else None
                    launch_dict['Version Booster'].append(bv)
                    
                    # Launch Site
                    launch_site = row[2].a.string if row[2].a else None
                    launch_dict['Launch site'].append(launch_site)
                    
                    # Payload
                    payload = row[3].a.string if row[3].a else None
                    launch_dict['Payload'].append(payload)
                    
                    # Payload Mass
                    payload_mass = get_mass(row[4])
                    launch_dict['Payload mass'].append(payload_mass)
                    
                    # Orbit
                    orbit = row[5].a.string if row[5].a else None
                    launch_dict['Orbit'].append(orbit)
                    
                    # Customer
                    customer = row[6].a.string if row[6].a else None
                    launch_dict['Customer'].append(customer)
                    
                    # Launch outcome
                    launch_outcome = list(row[7].strings)[0]
                    launch_dict['Launch outcome'].append(launch_outcome)
                    
                    # Booster landing
                    booster_landing = landing_status(row[8])
                    launch_dict['Booster landing'].append(booster_landing)
        
        # Convert the dictionary into a Pandas dataframe
        launch_df = pd.DataFrame(launch_dict)
        print(launch_df.head(200))
    else:
        print("The third table was not found.")
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")


Successfully fetched and parsed the page!
Page title: List of Falcon 9 and Falcon Heavy launches - Wikipedia
Column names: ['Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '14', '15', '16', '17', '18', '19', '20']
    Flight No. Date andtime (UTC) Version,Booster[b] Launch site Payload[c]  \
0            1               None               None       CCAFS       None   
1            2               None               None       CCAFS       None   
2            3               None               None       CCAFS       None   
3            4               None               None       CCAFS       None   
4            5               None               None       CCAFS       None   
..         ...                ...                ...         ...        ...   
116        117               None               None       CCSFS       None   
117        118               None          

In [96]:
launch_df.isnull().sum()

Flight No.              0
Date andtime (UTC)    121
Version,Booster[b]    121
Launch site             0
Payload[c]            121
Payload mass            0
Orbit                   0
Customer                2
Launchoutcome         121
Boosterlanding        121
14                    121
15                    121
16                    121
17                    121
18                    121
19                    121
20                    121
Payload                 0
Launch outcome          0
Version Booster         0
Booster landing         0
Date                    0
Time                    0
dtype: int64

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [97]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define the URL
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Fetch the HTML content using requests.get()
response = requests.get(static_url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    print("Successfully fetched and parsed the page!")
    
    # Print the page title to verify the BeautifulSoup object
    print("Page title:", soup.title.string)
    
    # Find all tables on the wiki page
    html_tables = soup.find_all('table', {'class': 'wikitable'})
    
    # Extract and print the content of the third table
    if len(html_tables) >= 3:
        first_launch_table = html_tables[2]
        
        # Define a function to extract column names from header
        def extract_column_from_header(header):
            return header.text.strip()
        
        # Extract column names from the third table
        column_names = []
        for header in first_launch_table.find_all('th'):
            name = extract_column_from_header(header)
            if name is not None and len(name) > 0:
                column_names.append(name)
        
        print("Column names:", column_names)
        
        # Create an empty dictionary with keys from the extracted column names
        launch_dict = dict.fromkeys(column_names)
        
        # Remove an irrelevant column
        if 'Date and time ( )' in launch_dict:
            del launch_dict['Date and time ( )']
        
        # Initialize the launch_dict with each value to be an empty list
        launch_dict['Flight No.'] = []
        launch_dict['Launch site'] = []
        launch_dict['Payload'] = []
        launch_dict['Payload mass'] = []
        launch_dict['Orbit'] = []
        launch_dict['Customer'] = []
        launch_dict['Launch outcome'] = []
        launch_dict['Version Booster'] = []
        launch_dict['Booster landing'] = []
        launch_dict['Date'] = []
        launch_dict['Time'] = []

        def date_time(cell):
            return cell.text.strip().split()

        def booster_version(cell):
            return cell.text.strip()

        def get_mass(cell):
            return cell.text.strip()

        def landing_status(cell):
            return cell.text.strip()
        
        extracted_row = 0
        # Extract each table
        for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
            # Get table row
            for rows in table.find_all("tr"):
                # Check to see if first table heading is a number corresponding to launch a number 
                if rows.th:
                    if rows.th.string:
                        flight_number = rows.th.string.strip()
                        flag = flight_number.isdigit()
                else:
                    flag = False
                # Get table element 
                row = rows.find_all('td')
                # If it is a number save cells in a dictionary 
                if flag:
                    extracted_row += 1
                    # Flight Number value
                    launch_dict['Flight No.'].append(flight_number)
                    
                    datatimelist = date_time(row[0])
                    
                    # Date value
                    date = datatimelist[0].strip(',')
                    launch_dict['Date'].append(date)
                    
                    # Time value
                    time = datatimelist[1]
                    launch_dict['Time'].append(time)
                    
                    # Booster version
                    bv = booster_version(row[1])
                    if not bv:
                        bv = row[1].a.string if row[1].a else None
                    launch_dict['Version Booster'].append(bv)
                    
                    # Launch Site
                    launch_site = row[2].a.string if row[2].a else None
                    launch_dict['Launch site'].append(launch_site)
                    
                    # Payload
                    payload = row[3].a.string if row[3].a else None
                    launch_dict['Payload'].append(payload)
                    
                    # Payload Mass
                    payload_mass = get_mass(row[4])
                    launch_dict['Payload mass'].append(payload_mass)
                    
                    # Orbit
                    orbit = row[5].a.string if row[5].a else None
                    launch_dict['Orbit'].append(orbit)
                    
                    # Customer
                    customer = row[6].a.string if row[6].a else None
                    launch_dict['Customer'].append(customer)
                    
                    # Launch outcome
                    launch_outcome = list(row[7].strings)[0]
                    launch_dict['Launch outcome'].append(launch_outcome)
                    
                    # Booster landing
                    booster_landing = landing_status(row[8])
                    launch_dict['Booster landing'].append(booster_landing)
        
        # Convert the dictionary into a Pandas dataframe
        df = pd.DataFrame({ key: pd.Series(value) for key, value in launch_dict.items() })
        print(df.head(50))
    else:
        print("The third table was not found.")
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")


Successfully fetched and parsed the page!
Page title: List of Falcon 9 and Falcon Heavy launches - Wikipedia
Column names: ['Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '14', '15', '16', '17', '18', '19', '20']
   Flight No.  Date andtime (UTC)  Version,Booster[b]     Launch site  \
0           1                 NaN                 NaN           CCAFS   
1           2                 NaN                 NaN           CCAFS   
2           3                 NaN                 NaN           CCAFS   
3           4                 NaN                 NaN           CCAFS   
4           5                 NaN                 NaN           CCAFS   
5           6                 NaN                 NaN            VAFB   
6           7                 NaN                 NaN           CCAFS   
7           8                 NaN                 NaN           CCAFS   
8           9     

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:137: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [87]:
df['Launchoutcome']

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
       ..
116   NaN
117   NaN
118   NaN
119   NaN
120   NaN
Name: Launchoutcome, Length: 121, dtype: float64

In [77]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define the static URL
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Fetch the HTML content using requests.get()
response = requests.get(static_url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    print("Successfully fetched and parsed the page!")
    
    # Print the page title to verify the BeautifulSoup object
    print("Page title:", soup.title.string)
    
    # Find all tables on the wiki page
    html_tables = soup.find_all('table', {'class': 'wikitable'})

    # Extract and print the content of the third table
    if len(html_tables) >= 3:
        first_launch_table = html_tables[2]

        # Extract column names from the third table
        column_names = []
        for header in first_launch_table.find_all('th'):
            name = header.text.strip()
            if name is not None and len(name) > 0:
                column_names.append(name)

        print("Column names:", column_names)

        # Define the correct column names
        corrected_column_names = {
            'Flight No.': 'Flight No.',
            'Date andtime (UTC)': 'Date and Time',
            'Version,Booster[b]': 'Version Booster',
            'Launch site': 'Launch site',
            'Payload[c]': 'Payload',
            'Payload mass': 'Payload mass',
            'Orbit': 'Orbit',
            'Customer': 'Customer',
            'Launchoutcome': 'Launch outcome',
            'Boosterlanding': 'Booster landing'
        }

        # Create an empty dictionary with corrected keys
        launch_dict = {corrected_column_names[key]: [] for key in column_names if key in corrected_column_names}

        # Include the missing 'Date' and 'Time' keys explicitly
        launch_dict['Date'] = []
        launch_dict['Time'] = []

        # Define helper functions for extracting data
        def date_time(cell):
            return cell.text.strip().split()

        def booster_version(cell):
            return cell.text.strip()

        def get_mass(cell):
            return cell.text.strip()

        def landing_status(cell):
            return cell.text.strip()
        
        extracted_row = 0
        # Extract each table
        for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
            # Get table row
            for rows in table.find_all("tr"):
                # Check to see if first table heading is a number corresponding to launch a number
                if rows.th:
                    if rows.th.string:
                        flight_number = rows.th.string.strip()
                        flag = flight_number.isdigit()
                else:
                    flag = False
                # Get table element
                row = rows.find_all('td')
                # If it is a number save cells in a dictionary
                if flag:
                    extracted_row += 1
                    # Flight Number value
                    launch_dict['Flight No.'].append(flight_number)

                    datatimelist = date_time(row[0])

                    # Date value
                    date = datatimelist[0].strip(',')
                    launch_dict['Date'].append(date)

                    # Time value
                    time = datatimelist[1] if len(datatimelist) > 1 else ''
                    launch_dict['Time'].append(time)

                    # Booster version
                    bv = booster_version(row[1])
                    if not bv:
                        bv = row[1].a.string if row[1].a else None
                    launch_dict['Version Booster'].append(bv)

                    # Launch Site
                    launch_site = row[2].a.string if row[2].a else None
                    launch_dict['Launch site'].append(launch_site)

                    # Payload
                    payload = row[3].a.string if row[3].a else None
                    launch_dict['Payload'].append(payload)

                    # Payload Mass
                    payload_mass = get_mass(row[4])
                    launch_dict['Payload mass'].append(payload_mass)

                    # Orbit
                    orbit = row[5].a.string if row[5].a else None
                    launch_dict['Orbit'].append(orbit)

                    # Customer
                    customer = row[6].a.string if row[6].a else None
                    launch_dict['Customer'].append(customer)

                    # Launch outcome
                    launch_outcome = list(row[7].strings)[0].strip()  # Remove newline characters
                    launch_dict['Launch outcome'].append(launch_outcome)

                    # Booster landing
                    booster_landing = landing_status(row[8])
                    launch_dict['Booster landing'].append(booster_landing)

        # Convert the dictionary into a Pandas dataframe
        df = pd.DataFrame({key: pd.Series(value) for key, value in launch_dict.items()})

        # Select only relevant columns
        relevant_columns = ['Flight No.', 'Date', 'Time', 'Version Booster', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Booster landing']
        df = df[relevant_columns]

        # Filter out Falcon 1 launches more carefully
        falcon_9_df = df[df['Version Booster'].str.contains('F9', na=False)]

        # Further filter for rows where Launch outcome is 'Success' and Version Booster does not contain '~V1.~'
        filtered_df = falcon_9_df[(falcon_9_df['Launch outcome'] == 'Success') & (~falcon_9_df['Version Booster'].str.contains('~V1.~', na=False))]

        # Count the number of filtered Falcon 9 launches
        filtered_falcon_9_launch_count = len(filtered_df)
        print(f"Number of filtered Falcon 9 launches: {filtered_falcon_9_launch_count}")
    else:
        print("The third table was not found.")
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")
print(df)

Successfully fetched and parsed the page!
Page title: List of Falcon 9 and Falcon Heavy launches - Wikipedia
Column names: ['Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '14', '15', '16', '17', '18', '19', '20']
Number of filtered Falcon 9 launches: 120
    Flight No. Date      Time       Version Booster Launch site  \
0            1    4      June  F9 v1.0[7]B0003.1[8]       CCAFS   
1            2    8  December  F9 v1.0[7]B0004.1[8]       CCAFS   
2            3   22       May  F9 v1.0[7]B0005.1[8]       CCAFS   
3            4    8   October  F9 v1.0[7]B0006.1[8]       CCAFS   
4            5    1     March  F9 v1.0[7]B0007.1[8]       CCAFS   
..         ...  ...       ...                   ...         ...   
116        117    9       May  F9 B5 ♺B1051.10[657]       CCSFS   
117        118   15       May   F9 B5 ♺B1058.8[660]         KSC   
118        119   26       May 

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:135: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [80]:

missing_values_count = df['Booster landing']
filtered_df =  df[df['Booster landing'].str.contains('NaN')]
print(filtered_df)

Empty DataFrame
Columns: [Flight No., Date, Time, Version Booster, Launch site, Payload, Payload mass, Orbit, Customer, Launch outcome, Booster landing]
Index: []


In [101]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define the static URL
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Fetch the HTML content using requests.get()
response = requests.get(static_url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    print("Successfully fetched and parsed the page!")
    
    # Print the page title to verify the BeautifulSoup object
    print("Page title:", soup.title.string)
    
    # Find all tables on the wiki page
    html_tables = soup.find_all('table', {'class': 'wikitable'})

    # Extract and print the content of the third table
    if len(html_tables) >= 3:
        first_launch_table = html_tables[2]

        # Extract column names from the third table
        column_names = []
        for header in first_launch_table.find_all('th'):
            name = header.text.strip()
            if name is not None and len(name) > 0:
                column_names.append(name)

        print("Column names:", column_names)

        # Define the correct column names
        corrected_column_names = {
            'Flight No.': 'Flight No.',
            'Date andtime (UTC)': 'Date and Time',
            'Version,Booster[b]': 'Version Booster',
            'Launch site': 'Launch site',
            'Payload[c]': 'Payload',
            'Payload mass': 'Payload mass',
            'Orbit': 'Orbit',
            'Customer': 'Customer',
            'Launchoutcome': 'Launch outcome',
            'Boosterlanding': 'Booster landing'
        }

        # Create an empty dictionary with corrected keys
        launch_dict = {corrected_column_names[key]: [] for key in column_names if key in corrected_column_names}

        # Include the missing 'Date' and 'Time' keys explicitly
        launch_dict['Date'] = []
        launch_dict['Time'] = []

        # Define helper functions for extracting data
        def date_time(cell):
            return cell.text.strip().split()

        def booster_version(cell):
            return cell.text.strip()

        def get_mass(cell):
            return cell.text.strip()

        def landing_status(cell):
            return cell.text.strip()
        
        extracted_row = 0
        # Extract each table
        for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
            # Get table row
            for rows in table.find_all("tr"):
                # Check to see if first table heading is a number corresponding to launch a number
                if rows.th:
                    if rows.th.string:
                        flight_number = rows.th.string.strip()
                        flag = flight_number.isdigit()
                else:
                    flag = False
                # Get table element
                row = rows.find_all('td')
                # If it is a number save cells in a dictionary
                if flag:
                    extracted_row += 1
                    # Flight Number value
                    launch_dict['Flight No.'].append(flight_number)

                    datatimelist = date_time(row[0])

                    # Date value
                    date = datatimelist[0].strip(',')
                    launch_dict['Date'].append(date)

                    # Time value
                    time = datatimelist[1] if len(datatimelist) > 1 else ''
                    launch_dict['Time'].append(time)

                    # Booster version
                    bv = booster_version(row[1])
                    if not bv:
                        bv = row[1].a.string if row[1].a else None
                    launch_dict['Version Booster'].append(bv)

                    # Launch Site
                    launch_site = row[2].a.string if row[2].a else None
                    launch_dict['Launch site'].append(launch_site)

                    # Payload
                    payload = row[3].a.string if row[3].a else None
                    launch_dict['Payload'].append(payload)

                    # Payload Mass
                    payload_mass = get_mass(row[4])
                    launch_dict['Payload mass'].append(payload_mass)

                    # Orbit
                    orbit = row[5].a.string if row[5].a else None
                    launch_dict['Orbit'].append(orbit)

                    # Customer
                    customer = row[6].a.string if row[6].a else None
                    launch_dict['Customer'].append(customer)

                    # Launch outcome
                    launch_outcome = list(row[7].strings)[0]
                    launch_dict['Launch outcome'].append(launch_outcome)

                    # Booster landing
                    booster_landing = landing_status(row[8])
                    launch_dict['Booster landing'].append(booster_landing)

        # Convert the dictionary into a Pandas dataframe
        df = pd.DataFrame({key: pd.Series(value) for key, value in launch_dict.items()})

        # Select only relevant columns
        relevant_columns = ['Flight No.', 'Date', 'Time', 'Version Booster', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Booster landing']
        df = df[relevant_columns]

        # Filter out Falcon 1 launches more carefully
        falcon_9_df = df[df['Version Booster'].str.contains('F9 v1', na=False)]
        print(falcon_9_df)
        # Count the number of Falcon 9 launches
        falcon_9_launch_count = len(falcon_9_df)
        print(f"Number of Falcon 9 launches: {falcon_9_launch_count}")
    else:
        print("The third table was not found.")
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")


Successfully fetched and parsed the page!
Page title: List of Falcon 9 and Falcon Heavy launches - Wikipedia
Column names: ['Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '14', '15', '16', '17', '18', '19', '20']
   Flight No. Date       Time       Version Booster     Launch site  \
0           1    4       June  F9 v1.0[7]B0003.1[8]           CCAFS   
1           2    8   December  F9 v1.0[7]B0004.1[8]           CCAFS   
2           3   22        May  F9 v1.0[7]B0005.1[8]           CCAFS   
3           4    8    October  F9 v1.0[7]B0006.1[8]           CCAFS   
4           5    1      March  F9 v1.0[7]B0007.1[8]           CCAFS   
5           6   29  September    F9 v1.1[7]B1003[8]            VAFB   
6           7    3   December          F9 v1.1B1004           CCAFS   
7           8    6    January               F9 v1.1           CCAFS   
8           9   18      April       

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:135: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [105]:
df.isnull().sum()
df[df['Boosterlanding ']]#.str.contains('None', na=False)]

KeyError: 'Boosterlanding '

In [29]:
import requests

# Define the URL for the SpaceX API
api_url = "https://api.spacexdata.com/v4/launches"

# Fetch the launch data from the API
response = requests.get(api_url)

# Check if the request was successful
if response.status_code == 200:
    launches = response.json()
    print("Successfully fetched launch data from the SpaceX API!")
    
    # Filter out Falcon 1 launches
    falcon_9_launches = [launch for launch in launches if 'Falcon 9' in launch['rocket']]
    
    # Count the number of Falcon 9 launches
    falcon_9_launch_count = len(falcon_9_launches)
    print(f"Number of Falcon 9 launches: {falcon_9_launch_count}")
else:
    print(f"Failed to retrieve launch data. Status code: {response.status_code}")


Successfully fetched launch data from the SpaceX API!
Number of Falcon 9 launches: 0


In [30]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define the URL
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Fetch the HTML content using requests.get()
response = requests.get(static_url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    print("Successfully fetched and parsed the page!")
    
    # Print the page title to verify the BeautifulSoup object
    print("Page title:", soup.title.string)
    
    # Find all tables on the wiki page
    html_tables = soup.find_all('table', {'class': 'wikitable'})
    
    # Extract and print the content of the third table
    if len(html_tables) >= 3:
        first_launch_table = html_tables[2]
        
        # Define a function to extract column names from header
        def extract_column_from_header(header):
            return header.text.strip()
        
        # Extract column names from the third table
        column_names = []
        for header in first_launch_table.find_all('th'):
            name = extract_column_from_header(header)
            if name is not None and len(name) > 0:
                column_names.append(name)
        
        print("Column names:", column_names)
        
        # Create an empty dictionary with keys from the extracted column names
        launch_dict = dict.fromkeys(column_names)
        
        # Remove an irrelevant column
        if 'Date andtime (UTC)' in launch_dict:
            del launch_dict['Date andtime (UTC)']
        
        # Initialize the launch_dict with each value to be an empty list
        for key in launch_dict:
            launch_dict[key] = []

        # Define helper functions for extracting data
        def date_time(cell):
            return cell.text.strip().split()

        def booster_version(cell):
            return cell.text.strip()

        def get_mass(cell):
            return cell.text.strip()

        def landing_status(cell):
            return cell.text.strip()
        
        extracted_row = 0
        # Extract each table
        for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
            # Get table row
            for rows in table.find_all("tr"):
                # Check to see if first table heading is a number corresponding to launch a number 
                if rows.th:
                    if rows.th.string:
                        flight_number = rows.th.string.strip()
                        flag = flight_number.isdigit()
                else:
                    flag = False
                # Get table element 
                row = rows.find_all('td')
                # If it is a number save cells in a dictionary 
                if flag:
                    extracted_row += 1
                    # Flight Number value
                    launch_dict['Flight No.'].append(flight_number)
                    
                    datatimelist = date_time(row[0])
                    
                    # Date value
                    date = datatimelist[0].strip(',')
                    launch_dict['Date'].append(date)
                    
                    # Time value
                    time = datatimelist[1]
                    launch_dict['Time'].append(time)
                    
                    # Booster version
                    bv = booster_version(row[1])
                    if not bv:
                        bv = row[1].a.string if row[1].a else None
                    launch_dict['Version,Booster[b]'].append(bv)
                    
                    # Launch Site
                    launch_site = row[2].a.string if row[2].a else None
                    launch_dict['Launch site'].append(launch_site)
                    
                    # Payload
                    payload = row[3].a.string if row[3].a else None
                    launch_dict['Payload[c]'].append(payload)
                    
                    # Payload Mass
                    payload_mass = get_mass(row[4])
                    launch_dict['Payload mass'].append(payload_mass)
                    
                    # Orbit
                    orbit = row[5].a.string if row[5].a else None
                    launch_dict['Orbit'].append(orbit)
                    
                    # Customer
                    customer = row[6].a.string if row[6].a else None
                    launch_dict['Customer'].append(customer)
                    
                    # Launch outcome
                    launch_outcome = list(row[7].strings)[0]
                    launch_dict['Launchoutcome'].append(launch_outcome)
                    
                    # Booster landing
                    booster_landing = landing_status(row[8])
                    launch_dict['Boosterlanding'].append(booster_landing)
        
        # Convert the dictionary into a Pandas dataframe
        df = pd.DataFrame({key: pd.Series(value) for key, value in launch_dict.items()})
        
        # Rename columns for better clarity
        df.rename(columns={
            'Version,Booster[b]': 'Version Booster',
            'Payload[c]': 'Payload',
            'Launchoutcome': 'Launch outcome',
            'Boosterlanding': 'Booster landing'
        }, inplace=True)
        
        # Select only relevant columns
        relevant_columns = ['Flight No.', 'Date', 'Time', 'Version Booster', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Booster landing']
        df = df[relevant_columns]
        
        print(df.head())
    else:
        print("The third table was not found.")
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")


Successfully fetched and parsed the page!
Page title: List of Falcon 9 and Falcon Heavy launches - Wikipedia
Column names: ['Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '14', '15', '16', '17', '18', '19', '20']


KeyError: 'Date'

In [31]:
import requests
import pandas as pd

# Define the URL for the SpaceX API
api_url = "https://api.spacexdata.com/v4/launches"

# Fetch the launch data from the API
response = requests.get(api_url)

# Check if the request was successful
if response.status_code == 200:
    launches = response.json()
    print("Successfully fetched launch data from the SpaceX API!")
    
    # Extract the relevant data into a DataFrame
    data = {
        'landingPad': [launch['cores'][0]['landpad'] if launch['cores'] and 'landpad' in launch['cores'][0] else None for launch in launches]
    }
    df = pd.DataFrame(data)
    
    # Count the number of missing values in the 'landingPad' column
    missing_values_count = df['landingPad'].isna().sum()
    print(f"Number of missing values in the 'landingPad' column: {missing_values_count}")
else:
    print(f"Failed to retrieve launch data. Status code: {response.status_code}")


Successfully fetched launch data from the SpaceX API!
Number of missing values in the 'landingPad' column: 54


In [32]:
import requests

# Define the URL for the SpaceX API
api_url = "https://api.spacexdata.com/v4/launches"

# Fetch the launch data from the API
response = requests.get(api_url)

# Check if the request was successful
if response.status_code == 200:
    launches = response.json()
    print("Successfully fetched launch data from the SpaceX API!")
    
    # Define the URL for the SpaceX rockets API to get the rocket IDs and names
    rockets_api_url = "https://api.spacexdata.com/v4/rockets"
    
    # Fetch the rocket data from the API
    rockets_response = requests.get(rockets_api_url)
    
    if rockets_response.status_code == 200:
        rockets = rockets_response.json()
        
        # Create a dictionary to map rocket IDs to rocket names
        rocket_id_to_name = {rocket['id']: rocket['name'] for rocket in rockets}
        
        # Filter out Falcon 1 launches
        falcon_9_launches = [
            launch for launch in launches 
            if rocket_id_to_name.get(launch['rocket']) == 'Falcon 9'
        ]
        
        # Count the number of Falcon 9 launches
        falcon_9_launch_count = len(falcon_9_launches)
        print(f"Number of Falcon 9 launches: {falcon_9_launch_count}")
    else:
        print(f"Failed to retrieve rocket data. Status code: {rockets_response.status_code}")
else:
    print(f"Failed to retrieve launch data. Status code: {response.status_code}")


Successfully fetched launch data from the SpaceX API!
Number of Falcon 9 launches: 195


In [33]:
import requests
import pandas as pd

# Define the URL for the SpaceX API
api_url = "https://api.spacexdata.com/v4/launches"

# Fetch the launch data from the API
response = requests.get(api_url)

# Check if the request was successful
if response.status_code == 200:
    launches = response.json()
    print("Successfully fetched launch data from the SpaceX API!")
    
    # Extract the relevant data into a DataFrame
    data = {
        'landingPad': [core['landpad'] if core and 'landpad' in core else None for launch in launches for core in launch['cores']]
    }
    df = pd.DataFrame(data)
    
    # Count the number of missing values in the 'landingPad' column
    missing_values_count = df['landingPad'].isnull().sum()
    print(f"Number of missing values in the 'landingPad' column: {missing_values_count}")
else:
    print(f"Failed to retrieve launch data. Status code: {response.status_code}")


Successfully fetched launch data from the SpaceX API!
Number of missing values in the 'landingPad' column: 58


In [56]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define the static URL
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Fetch the HTML content using requests.get()
response = requests.get(static_url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    print("Successfully fetched and parsed the page!")
    
    # Print the page title to verify the BeautifulSoup object
    print("Page title:", soup.title.string)
    
    # Find all tables on the wiki page
    html_tables = soup.find_all('table', {'class': 'wikitable'})

    # Extract and print the content of the relevant table
    if len(html_tables) >= 3:
        first_launch_table = html_tables[2]

        # Extract column names from the third table
        column_names = []
        for header in first_launch_table.find_all('th'):
            name = header.text.strip()
            if name is not None and len(name) > 0:
                column_names.append(name)

        print("Column names:", column_names)

        # Define the correct column names
        corrected_column_names = {
            'Flight No.': 'Flight No.',
            'Date andtime (UTC)': 'Date and Time',
            'Version,Booster[b]': 'Version Booster',
            'Launch site': 'Launch site',
            'Payload[c]': 'Payload',
            'Payload mass': 'Payload mass',
            'Orbit': 'Orbit',
            'Customer': 'Customer',
            'Launchoutcome': 'Launch outcome',
            'Boosterlanding': 'Booster landing'
        }

        # Create an empty dictionary with corrected keys
        launch_dict = {corrected_column_names[key]: [] for key in column_names if key in corrected_column_names}

        # Include the missing 'Date' and 'Time' keys explicitly
        launch_dict['Date'] = []
        launch_dict['Time'] = []

        # Define helper functions for extracting data
        def date_time(cell):
            return cell.text.strip().split()

        def booster_version(cell):
            return cell.text.strip()

        def get_mass(cell):
            return cell.text.strip()

        def landing_status(cell):
            return cell.text.strip()
        
        extracted_row = 0
        # Extract each table
        for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
            # Get table row
            for rows in table.find_all("tr"):
                # Check to see if first table heading is a number corresponding to launch a number
                if rows.th:
                    if rows.th.string:
                        flight_number = rows.th.string.strip()
                        flag = flight_number.isdigit()
                else:
                    flag = False
                # Get table element
                row = rows.find_all('td')
                # If it is a number save cells in a dictionary
                if flag:
                    extracted_row += 1
                    # Flight Number value
                    launch_dict['Flight No.'].append(flight_number)

                    datatimelist = date_time(row[0])

                    # Date value
                    date = datatimelist[0].strip(',')
                    launch_dict['Date'].append(date)

                    # Time value
                    time = datatimelist[1] if len(datatimelist) > 1 else ''
                    launch_dict['Time'].append(time)

                    # Booster version
                    bv = booster_version(row[1])
                    if not bv:
                        bv = row[1].a.string if row[1].a else None
                    launch_dict['Version Booster'].append(bv)

                    # Launch Site
                    launch_site = row[2].a.string if row[2].a else None
                    launch_dict['Launch site'].append(launch_site)

                    # Payload
                    payload = row[3].a.string if row[3].a else None
                    launch_dict['Payload'].append(payload)

                    # Payload Mass
                    payload_mass = get_mass(row[4])
                    launch_dict['Payload mass'].append(payload_mass)

                    # Orbit
                    orbit = row[5].a.string if row[5].a else None
                    launch_dict['Orbit'].append(orbit)

                    # Customer
                    customer = row[6].a.string if row[6].a else None
                    launch_dict['Customer'].append(customer)

                    # Launch outcome
                    launch_outcome = list(row[7].strings)[0].strip()  # Remove newline characters
                    launch_dict['Launch outcome'].append(launch_outcome)

                    # Booster landing
                    booster_landing = landing_status(row[8])
                    launch_dict['Booster landing'].append(booster_landing)

        # Convert the dictionary into a Pandas dataframe
        df = pd.DataFrame({key: pd.Series(value) for key, value in launch_dict.items()})

        # Select only relevant columns
        relevant_columns = ['Flight No.', 'Date', 'Time', 'Version Booster', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Booster landing']
        df = df[relevant_columns]

        # Filter out Falcon 1 launches more carefully
        falcon_9_df = df[df['Version Booster'].str.contains('Falcon 9', na=False)]

        # Further filter for rows where Launch outcome is 'Success' and Version Booster does not contain '~V1.~'
        filtered_df = falcon_9_df[(falcon_9_df['Launch outcome'] == 'Success') & (~falcon_9_df['Version Booster'].str.contains('~V1.~', na=False))]

        # Count the number of filtered Falcon 9 launches
        filtered_falcon_9_launch_count = len(filtered_df)
        print(f"Number of filtered Falcon 9 launches: {filtered_falcon_9_launch_count}")
    else:
        print("The third table was not found.")
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")


Successfully fetched and parsed the page!
Page title: List of Falcon 9 and Falcon Heavy launches - Wikipedia
Column names: ['Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '14', '15', '16', '17', '18', '19', '20']
Number of filtered Falcon 9 launches: 0


/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:135: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |


Copyright © 2021 IBM Corporation. All rights reserved.
